# 실습에 필요한 패키지 설치

- LangChain 관련 : langchain, langchain-openai, langchain-community
- Document Loading 관련: pypdf, pymupdf, arxiv
- Document Embedding 관련: sentense-transformers
- Vector Store 관련: chromadb, faiss-cpu 등

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install -qU arxiv
!pip install -U sentence-transformers
!pip install chromadb
!pip install -q pypdf pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 37.6 MB/s eta 0:00:00


* poppler-utils: PDF 파일을 조작하고 다른 형식으로 변환하기 위한 사전 컴파일된 명령줄 유틸리티

In [ ]:
!apt-get install -y poppler-utils

* 실습 데이터 준비

In [ ]:
!git clone https://github.com/tsdata/langchain-study.git

In [ ]:
%cd langchain-study/data

In [ ]:
import os
import bs4
import numpy as np
from numpy import dot
from numpy.linalg import norm
from glob import glob
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.document_loaders import (
    WebBaseLoader,
    TextLoader,
    DirectoryLoader,
    CSVLoader,
    PyPDFLoader, PyMuPDFLoader, OnlinePDFLoader, PyPDFDirectoryLoader
)
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv() 
openai_api_key = os.getenv("OPENAI_API_KEY")

# DataLoader

* WebBaseLoader : 특정 웹 페이지의 내용을 로드하고 파싱
* TextLoader : 텍스트 파일을 불러옴
* DirectoryLoader : 디렉토리 내 모든 문서를 가져옴
* CSVLoader : csv 파일 데이터를 가져옴.
* PDFLoader
  * PyPDFLoader : PDF 파일에서 텍스트 추출
  * UnstructuredPDFLoader : 형식이 없는 PDF 문서 로드
  * PyMuPDFLoader : 메타 데이터를 상세하게 추출
  * OnlinePDFLoader : 온라인 PDF 파일 로드
  * PyPDFDirectoryLoader : 특정 폴더의 모든 PDF 문서 로드


In [ ]:
# WebBaseLoader
url1 = "https://blog.langchain.dev/week-of-6-10-langchain-release-notes/"
url2 = "https://blog.langchain.dev/week-of-2-5-24-langchain-release-notes/"
url3 = "https://www.skelterlabs.com/blog/2024-year-of-the-rag/"

#bs4.SoupStrainer를 사용하여 특정 클래스 이름을 가진 HTML 요소만 선택하여 파싱
loader = WebBaseLoader(
    web_paths=(url1, url2),
    encoding="utf-8",
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("article-header", "article-content")
        )
    ),
)

docs = loader.load()
len(docs)

* 일반 Text Loader : *.txt 형식

---



In [ ]:
# TextLoader
from langchain_community.document_loaders import TextLoader

loader = TextLoader('history.txt')
data = loader.load()

print(type(data))
print(len(data))


In [ ]:
len(data[0].page_content)
data[0].metadata

* Directory 내 특정 확장자를 갖는 파일 로드
 - glob()을 이용하여 대상지정
 - DirectoryLoader()

---



In [ ]:
files = glob(os.path.join('./', '*.txt'))
files

* CSV 확장자를 갖는 데이터 로드: CSVLoader()
  - default loading
  - 데이터 출처 정보를 특정 필드(열, column)로 지정: source_column 속성 설정
  - CSV 파싱 옵션을 지정
---



In [ ]:
loader = CSVLoader(file_path='한국주택금융공사_주택금융관련_지수_20160101.csv', encoding='cp949')
data = loader.load()

len(data)
data[0]


In [ ]:
# source_column 속성에 데이터의 출처 정보로 사용될 열의 이름을 지정할 수 있다.
loader = CSVLoader(file_path='한국주택금융공사_주택금융관련_지수_20160101.csv', encoding='cp949',
                   source_column='연도')

data = loader.load()

data[0]

In [ ]:
#기존에 콤마가 아니라 구분자를 \n으로 봄.
loader = CSVLoader(file_path='한국주택금융공사_주택금융관련_지수_20160101.csv', encoding='cp949',
                   csv_args={
                       'delimiter': '\n',
                   })

data = loader.load()

data[0]


 - PyPDFLoader
    - PDF 문서 페이지별로 로드, 텍스트를 추출하여 documents list 객체로 반환

 - PyMuPDFLoader

    - PDF 파일의 페이지를 로드하고, 각 페이지를 개별 document 객체로 추출
    - 자세한 메타데이터 추출도 가능

 - PyPDFDirectoryLoader
   - 특정 폴더에 있는 모든 PDF파일을 가져옴

---



---



In [ ]:
pdf_filepath = '000660_SK_2023.pdf'
loader = PyPDFLoader(pdf_filepath)
pages = loader.load()

print(len(pages))
pages[10]

In [ ]:
#pdf_filepath = '/content/langchain-study/data/300720_한일시멘트_2023.pdf'

pdf_filepath = '000660_SK_2023.pdf'
loader = PyMuPDFLoader(pdf_filepath)
pages = loader.load()

len(pages)


In [ ]:
pages[0].page_content


In [ ]:
pages[0].metadata


* ArxivLoader를 사용하여 arXiv에서 논문을 로드
    - query: 매개변수에 arXiv ID "1605.08386"을 전달하여 특정 논문을 검색
    - load_max_docs: 매개변수를 2로 설정하여 최대 2개의 논문을 로드.

---    

In [ ]:
!pip install -qU arxiv

In [ ]:
# ArxivLoader를 사용하여 arXiv에서 문서를 로드합니다. query 매개변수는 검색할 논문의 arXiv ID이고, load_max_docs 매개변수는 로드할 최대 문서 수를 지정합니다.
from langchain_community.document_loaders import ArxivLoader

#docs = ArxivLoader(query="2005.11401", load_max_docs=5).load()
docs = ArxivLoader(query="RAG", load_max_docs=5).load()
print(len(docs))  # 로드된 문서의 개수를 반환합니다.
docs[0]

# TextSplit

* CharacterTextSplitter : 텍스트를 문자 단위로 분할하는 데 사용되는 Class
* RecursiveCharacterTextSplitter : 텍스트를 재귀적으로 분할하는 Class

In [ ]:
loader = TextLoader('history.txt')
data = loader.load()

print(len(data[0].page_content))
data[0].page_content

In [ ]:
text_splitter = CharacterTextSplitter(
    separator = '',
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(data[0].page_content)

len(texts)

In [ ]:
texts[0]

In [ ]:
text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(data[0].page_content)

len(texts)

In [ ]:
texts[0]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(data[0].page_content)

len(texts)

In [ ]:
texts[0]

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)

docs = text_splitter.split_documents(data)

len(docs)
print(len(docs[0].page_content))

# Embeddings

* OpenAIEmbeddings
  * OpenAI API를 활용하여, 각 문서를 대응하는 임베딩 벡터로 변환
* HuggingFaceEmbeddings
  * HuggingFace 모델에서 사용된 사전 훈련된 임베딩 모델을 다운로드 받아서 적용 가능
* GoogleGenerativeAIEmbeddings
  * Google 생성형 AI 모델을 활용하여 문서나 문장을 임베딩할 수 있음.

In [ ]:
embeddings_model = OpenAIEmbeddings()

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')



In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

In [ ]:
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')


In [ ]:
for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

## Vector Store

 * embedding은 OpenAIEmbedding 사용

* Chroma
* Chroma MMR

  

In [ ]:
loader = TextLoader('history.txt')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=50,
    encoding_name='cl100k_base'
)

texts = text_splitter.split_text(data[0].page_content)

embeddings_model = OpenAIEmbeddings()
db = Chroma.from_texts(
    texts,
    embeddings_model,
    collection_name = 'history',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

query = '누가 한글을 창제했나요?'
docs = db.similarity_search(query)


print(docs[0].page_content)

mmr_docs = db.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

 * Chroma MMR

In [ ]:
loader =  PyMuPDFLoader('/content/langchain-study/data/300720_한일시멘트_2023.pdf') #PDF 파일에서 텍스트 데이터를 로드

data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(data)

embeddings_model = OpenAIEmbeddings()
db2 = Chroma.from_documents(
    documents,
    embeddings_model,
    collection_name = 'pdf',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

query = '한일시멘트 가격관련 이슈를 알려줘'
mmr_docs = db2.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

 * FAISS
   -

In [ ]:
!pip install -qU langchain-community faiss-cpu

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
loader = PyMuPDFLoader('/content/langchain-study/data/300720_한일시멘트_2023.pdf') #PDF 파일에서 텍스트 데이터를 로드
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'   # 텍스트를 토큰으로 변환하는 인코딩 방식(‘cl100k_base’: ada-002 model 사용)
)

documents = text_splitter.split_documents(data)

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)


vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )
query = '한일시멘트의 2023년 친환경정책 관련 이슈는?'
docs = vectorstore.similarity_search(query)
print(len(docs))
print(docs[0].page_content)

### FAISS MMR

In [ ]:
mmr_docs = vectorstore.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

## Vector Store Retriever



In [ ]:
loader = PyMuPDFLoader('/content/langchain-study/data/300720_한일시멘트_2023.pdf') #PDF 파일에서 텍스트 데이터를 로드
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'
) # 분할하는 인스턴스를 생성

documents = text_splitter.split_documents(data) # chunk 단위로 분할

In [ ]:
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 1}) #벡터스토어에 Retriever 객체를 생성

In [ ]:
query = '한일시멘트의 2023년 친환경 정책은 무엇인가? '
docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[0]

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'fetch_k': 50}
)

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[2]

In [ ]:
# threshold를 기준으로 문서 추출
retriever = vectorstore.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.1}
)

docs = retriever.get_relevant_documents(query)
print(len(docs))

In [ ]:
# 문서 객체의 metadata를 이용한 필터링
retriever = vectorstore.as_retriever(
    search_kwargs={'filter': {'format':'PDF 1.7'}}
)

docs = retriever.get_relevant_documents(query)
print(len(docs))
#docs[0]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)

docs = retriever.get_relevant_documents(query)

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    max_tokens=500,
)


def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = prompt | llm | StrOutputParser()

# Run
response = chain.invoke({'context': (format_docs(docs)), 'question':query})
response

## MultiQueryRetriever

- 벡터스토어 검색도구(Vector Store Retriever)의 한계를 극복하기 위해 고안된 방법

- 단일 쿼리의 의미를 다양한 관점으로 확장하여 멀티 쿼리를 자동 생성하고, 이러한 모든 쿼리에 대한 검색 결과를 결합하여 처리

-  다양한 문장을 생성하기 위하여 LLM을 사용하여 사용자의 입력 문장을 다양한 관점으로 패러프레이징(Paraphrasing)하는 방식으로 구현

---
< 실행 방법 >

- MultiQueryRetriever 설정
  - from_llm 메서드를 통해, 기존 벡터저장소 검색도구(vectorstore.as_retriever())와 LLM 모델(llm)을 결합하여 MultiQueryRetriever 인스턴스를 생성합니다. 이때 LLM은 다양한 관점의 쿼리를 생성하는 데 사용됩니다.

- 로깅 설정
  - 로깅을 설정하여 MultiQueryRetriever에 의해 생성되고 실행되는 쿼리들에 대한 정보를 로그로 기록하고 확인할 수 있습니다. 검색 과정에서 어떤 쿼리들이 생성되고 사용되었는지 이해하는 데 도움이 됩니다.

- 문서 검색 실행
  - get_relevant_documents 메서드를 사용하여 주어진 사용자 쿼리(question)에 대해 멀티 쿼리 기반의 문서 검색을 실행합니다. 생성된 모든 쿼리에 대해 문서를 검색하고, 중복을 제거하여 고유한 문서들만을 결과로 반환합니다.




In [ ]:
question = '한일시멘트의 친환경 정책에 대해 알려줘'

llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    max_tokens=500,
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)
unique_docs[1]

In [ ]:
# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = (
    {'context': retriever_from_llm | format_docs, 'question': RunnablePassthrough()} #사용자의 질문을 그대로 전달
    | prompt
    | llm
    | StrOutputParser()
)

# Run
response = chain.invoke('한일시멘트의 친환경 정책에 대해 알려줘')
response

## Contextual Compression

 문서 압축기는 기본 검색기로부터 얻은 문서들을 더욱 효율적으로 압축하여, 쿼리와 가장 관련이 깊은 내용만을 추려내는 것을 목표로 하고, LLMChainExtractor와 ContextualCompressionRetriever 클래스를 사용합니다.

---
<실행 방법>

* LLMChainExtractor 설정

 - LLMChainExtractor.from_llm(llm)를 사용하여 문서 압축기를 설정.
 - 언어 모델(llm)을 사용하여 문서 내용을 압축

* ContextualCompressionRetriever 설정

  - ContextualCompressionRetriever 인스턴스를 생성할 때, base_compressor와 base_retriever를 인자로 제공.

  - base_compressor는 앞서 설정한 LLMChainExtractor 인스턴스이며, base_retriever는 기본 검색기 인스턴스.
  - 이 두 구성 요소를 결합하여 검색된 문서들을 압축하는 과정을 처리.

* 압축된 문서 검색
 - compression_retriever.get_relevant_documents(question) 함수를 사용하여 주어진 쿼리에 대한 압축된 문서들을 검색
 - 기본 검색기를 통해 얻은 문서들을 문서 압축기를 사용하여 내용을 압축하고, 쿼리와 가장 관련된 내용만을 추려냄

* 결과 출력



In [ ]:
question = '한일시멘트의 친환경 정책에 대해 알려줘.'

llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    max_tokens=500,
)

base_retriever = vectorstore.as_retriever(
                                search_type='mmr',
                                search_kwargs={'k':7, 'fetch_k': 20})

docs = base_retriever.get_relevant_documents(question)
print(len(docs))

In [ ]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retriever
)

compressed_docs = compression_retriever.get_relevant_documents(question)
print(len(compressed_docs))

In [ ]:
compressed_docs

In [ ]:
# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in compressed_docs])

# Chain
chain = (
    {'context': retriever_from_llm | format_docs, 'question': RunnablePassthrough()} #사용자의 질문을 그대로 전달
    | prompt
    | llm
    | StrOutputParser()
)

# Run
response = chain.invoke('한일시멘트의 친환경 정책에 대해 알려줘')
response